In [7]:
# -*- coding: utf-8 -*-
import asyncio
from twikit import Client  # No need to write "from twikit.twikit_async import Client" 
import pandas as pd
from datetime import datetime, timedelta
import time


#Se necesita usar la librería twikit para poder hacer el scrapping de los tweets. Para instalar, basta con usar: pip install twikit.
#La versión de python que funciona para usar esta librería es la 3.11 o posteriores.

"""Twikit es un modulo de python desarrollado por personas independientes. Este modulo lo que hace es 'simular' lo que haría una persona para adquirir los datos de X (anteriormente llamado twitter) SIN USAR LA API: toma, por ejemplo, los datos personales para inicio de sesión y, valga la redundancia, con ello abre una sesión de X y luego ahí sí puede capturar los tweets o datos según se le indique. También se puede hacer uso de las cookies para no tener que ingresar los datos de inicio de sesión. EXISTE RIESGO DE SUSPENSIÓN DE CUENTA EN CASO TAL DE INCUMPLIR CON LAS NORMAS DE LA RED SOCIAL, así que toca usarlo sabiamente, por ejemplo: scrappeando los tweets en intervalos de tiempo lejanos (yo he utilizado cada 15 o 30 minutos durante 8-9 horas y sin sobrepasarme de los 300 tweets al dia que X permite visualizar para un usuario nuevo y no verificado)"""


#NOTA: El método "created_at_datetime" está en utc ± 0, por lo tanto, toca restar 5 horas a la fecha por lo de arriba

"Twikit es un modulo de python desarrollado por personas independientes. Este modulo lo que hace es 'simular' lo que haría una persona para adquirir los datos de X (anteriormente llamado twitter) SIN USAR LA API: toma, por ejemplo, los datos personales para inicio de sesión y, valga la redundancia, con ello abre una sesión de X y luego ahí sí puede capturar los tweets o datos según se le indique. También se puede hacer uso de las cookies para no tener que ingresar los datos de inicio de sesión. EXISTE RIESGO DE SUSPENSIÓN DE CUENTA EN CASO TAL DE INCUMPLIR CON LAS NORMAS DE LA RED SOCIAL, así que toca usarlo sabiamente, por ejemplo: scrappeando los tweets en intervalos de tiempo lejanos (yo he utilizado cada 15 o 30 minutos durante 8-9 horas y sin sobrepasarme de los 300 tweets al dia que X permite visualizar para un usuario nuevo y no verificado)"

In [8]:
#Datos de inicio de sesión de X (Todo va en string)
USERNAME = ""
EMAIL = ""
PASSWORD = ""

#Se debe inicializar el cliente

client = Client('es-MX')

await client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD)

client.save_cookies('cookies.json') #Guarda las cookies para que no tenga que volver a iniciar sesión en la cuenta X y así evitar ser suspendida.


In [9]:

async def main():
    client.load_cookies('cookies.json') #Carga las cookies anteriormente guardadas. 

    print ('Ejecutando Script...') #Un print controlador: solo sirve para saber si se está ejecutando la función. Es opcional.
    tweets = await client.search_tweet('#Alberto OR #Monterrey OR #NuevoLeon OR #NuevoLeón OR #inundaciones OR Monterrey','Latest', count = 20) #Variable que busca los tweets con el metodo ".search_tweet" de la librería. El parametro "count" es la cantidad de tweets a recolectar. El límite de tweets es de 20. Solo se pueden buscar los tweets que están en la sección "Top", "Latest" o "Media". Se pueden concatenar terminos con el operador OR dentro del string. until:2024-6-26
    more_tweets = await tweets.next() #Variable que ayuda a buscar más tweets aparte de los que se buscan con el método ".search_tweets". Se pueden concatenar, así como lo hice en la linea. Sin embargo, también tiene el limite de 20 tweets y maximo 50 tweets por cada 15 minutos. Si se excede ese limite, el metodo no funciona y por lo tanto el bloque de codigo ya no sirve. Toca esperar a que se actualice.
    
    #for tweet in tweets:
    #    print("-",
    #        tweet.user.name,
    #        tweet.text,
    #        tweet.created_at_datetime - timedelta(hours=5), #Se resta 5 horas para cuadrar la zona horaria
    #    )

    final_tweets = [] #Lista donde guardaremos los tweets con sus datos de usuario y fecha
    for tweet in tweets:
        data = [tweet.user.name, tweet.text, (tweet.created_at_datetime - timedelta(hours=5)).strftime("%d/%m/%Y %H:%M:%S")] #Lista de datos: usuario, texto del tweet y fecha en formato "%d/%m/%Y %H:%M:%S". El "- timedelta(hours=5)" es por la nota del inicio del código.
        final_tweets.append(data) #Se añade los datos a la lista
    
    #Se hace lo mismo que anteriormente hicimos, pero en este caso, para la variable que nos recolecta más tweets
    for tweet in more_tweets:
        data = [tweet.user.name, tweet.text, (tweet.created_at_datetime - timedelta(hours=5)).strftime("%d/%m/%Y %H:%M:%S")]
        final_tweets.append(data) 

    #Cargamos el excel
    df_excel = pd.read_excel("Base.xlsx")

    #Guardamos los ultimos tweets de arriba
    data = pd.DataFrame(final_tweets, columns = ["User", "Text", "Date"])

    #Concatenamos
    Excel = pd.concat([df_excel, data])

    #Eliminamos duplicados 
    df_sin_duplicados = Excel.drop_duplicates()

    #Creamos el archivo
    df_sin_duplicados.to_excel("Base.xlsx", index = False) #Guardo en formato excel para luego, desde el propio excel, exportarlo. Me genera "menos inconveniente" hacerlo así. 
    time.sleep(600) #Linea que sirve para congelar el bloque de codigo que ayuda a scrappear los tweets. Esto se hace para recolectar los tweets cada t segundos y así evitar que: 1) el bucle genere conflicto a la hora de ejecutar la función que añade al excel los datos y 2) Nos suspendan la cuenta. #Recomiendo que se recolecte cada 15 o 30 minutos.


In [ ]:
#Contador auxiliar que nos ayuda a interrumpir el bucle infinito 
contador = 0


#Bucle para no tener que manualmente estar ejecutando el script
while True: 
    print("El contador va en:", contador) #Print que ayuda a saber en qué iteración va y también nos sirve para hacer calculo mental de cuantos tweets aproximadamente se pudieron haber scrappeado hasta el momento
    await main() #Se ejecuta la función para recolectar los tweets y guardarlos en el excel
    contador += 1
    if contador == 5: #Numero de veces que queramos hacer la recolección de tweets. Por ejemplo, si quiero recolectar 10 veces, entonces coloco 10. Si todo sale bien, eso serían entonces 20 tweets x 10 = 200 tweets cada t segundos que haya colocado arriba en el "time.sleep()".
        break


El contador va en: 0
Ejecutando Script...
El contador va en: 1
Ejecutando Script...
El contador va en: 2
Ejecutando Script...
El contador va en: 3
Ejecutando Script...
El contador va en: 4
Ejecutando Script...
